# This notebook checks whether we've added section_modes and section_distances to all confirmed trips in the database

In [ ]:
import sys
import time
import pandas as pd
from turtle import update
sys.path.insert(0,"/Users/mallen2/alternate_branches/eval-compatible-server/e-mission-server") 
#input("Enter your path to the emission server: ") )   # maybe I could try an emission import and then do this as a catch?
import emission.storage.decorations.trip_queries as esdtq
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.builtin_timeseries as estbt
import emission.core.wrapper.entry as ecwe
import emission.core.get_database as edb

start = 227

In [ ]:
print("Fetching user ids.")
user_list = []
# This info is in the Stage_uuids collection of the database
for u in edb.get_uuid_db().find():         # add users to proper locations in programs 
    uuid = u["uuid"]
    # Append the user to the list.
    user_list.append(uuid)

gis_sensed_modes = {0 : 'no_sensed',    # UNKNOWN  #NOTE: this is important info to mention.
    1 : 'walking',    # WALKING
    2 : 'bicycling',    # BICYCLING
    3 : 'bus',        # BUS
    4 : 'train',      # TRAIN
    5 : 'car',        # CAR
    6 : 'air_or_hsr', # AIR_OR_HSR
    7 : 'subway',      # SUBWAY
    8 : 'train',      # TRAM
    9 : 'train',      # LIGHT_RAIL
}

print("Fetching confirmed trips for each user.")
t1 = time.time()
t_first = time.time()
confirmed_trip_df_map = {}
i = start
n_missing = 0
for u in user_list[start:]:
    ts = esta.TimeSeries.get_time_series(u)
    result_it = ts.find_entries(["analysis/confirmed_trip"],time_query = None, geo_query = None,
                extra_query_list=None)
    # ct_df = ts.get_data_df("analysis/confirmed_trip")
    # get list of sections.
    # get list of trips.
    # if they have the same, then merge.
    for ct in result_it:
        # Only query and update if we need to.
        if not set(['section_modes', 'section_distances']).issubset(ct['data'].keys()):
            print(f"Participant {i} doesn't have section info yet.")
            n_missing += 1
            break
        print(ct['data']['section_modes'])
        
    print(f"{i} df found and updated in {time.time()-t_first} seconds.")
    i += 1
print(f"Fetching trips took {time.time()-t1} seconds.")
print(f"{n_missing} participants need sections found.")